# Search-Engine-from-Scratch

# NAME(s) and NIA(s) 

**Author :** Stephen Appiah.
         Umar Muhammad.
**NIA :** 206637.
      208244.
    

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
%cd /content/drive/Shareddrives/Information Retrieval and Web Analytics/Final Lab/P1-Modeling and Indexing_students/IRWA-Final-Project

/content/drive/Shareddrives/Information Retrieval and Web Analytics/Final Lab/P1-Modeling and Indexing_students/IRWA-Final-Project


In [35]:
!pwd
!ls

/content/drive/Shareddrives/Information Retrieval and Web Analytics/Final Lab/P1-Modeling and Indexing_students/IRWA-Final-Project
inputs	LICENSE  README.md  Search_Engine_From_Scratch.ipynb


#### Load Python packages
Let's first import all the packages that you will need during this assignment.

In [36]:
# if you do not have 'nltk', the following command should work "python -m pip install nltk"
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
import time
from numpy import linalg as la

#### Load data into memory
Import the 500 Wikipedia articles (one article per line). For each article we have the document id, document title and document body separated by "|" character.

In [38]:
#Read JSON ata set
def read_data(docs_path):
  import json
  # Opening JSON file
  f = open(docs_path)
  # returns JSON object as
  # a dictionary
  data = json.load(f)
  f.close()
  return data

data = read_data('inputs/dataset_tweets_WHO.json')

In [83]:
#Data Display
data.get('1')


{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Oct 13 08:46:17 +0000 2021',
 'display_text_range': [0, 241],
 'entities': {'hashtags': [{'indices': [0, 8], 'text': 'COVID19'}],
  'media': [{'display_url': 'pic.twitter.com/jpUQpnu0V1',
    'expanded_url': 'https://twitter.com/WHO/status/1448208458604584960/photo/1',
    'id': 1448208218463875077,
    'id_str': '1448208218463875077',
    'indices': [242, 265],
    'media_url': 'http://pbs.twimg.com/media/FBkRs6ZXMAUmafc.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/FBkRs6ZXMAUmafc.jpg',
    'sizes': {'large': {'h': 1152, 'resize': 'fit', 'w': 2048},
     'medium': {'h': 675, 'resize': 'fit', 'w': 1200},
     'small': {'h': 383, 'resize': 'fit', 'w': 680},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'type': 'photo',
    'url': 'https://t.co/jpUQpnu0V1'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'extended_entities': {'media': [{'display_url': 'pic.twitter.com/jpUQpnu0V1',
    'ex

In [40]:
print("Total number of Tweets in the corpus: {}".format(len(data)))

Total number of Tweets in the corpus: 2399


# Building the Index


Processing the data

Define functions to clean the text (tokenization / bigram/ lowercase /remove punctuation /stemming)

In [51]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def remove_stopwords(words):
    return [w for w in words if w.lower() not in stop_words]

def stem_each(words):
    return [stemmer.stem(w) for w in words]

def remove_white_space(text):
    return ' '.join(text.split())

In [52]:
def preprocess(text):
  """
  Preprocess the article text (title + body) removing stop words, stemming,
  transforming in lowercase and return the tokens of the text.

  Argument:
  line -- string (text) to be preprocessed

  Returns:
  line - a list of tokens corresponding to the input text after the preprocessing
  """
  stemmer = PorterStemmer()
  stop_words = set(stopwords.words("english"))
  text=  text.lower()
  text = text.replace('#', '')
  text=  text.split()
  text= remove_stopwords(text)
  text= stem_each(text)
  return text

In [55]:
#Eample
tweet = data.get('0')
print('full_text', tweet.get('full_text'))
print()
#phrase = "Smart-assistants like Apple’s Siri and Amazon’s Alexa recognize patterns in speech thanks to voice recognition, then infer meaning and provide a useful response."
print(preprocess(tweet.get('full_text')))

full_text It's International Day for Disaster Risk Reduction

#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.

Start learning today &amp; be #Ready4Response:
👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS

['intern', 'day', 'disast', 'risk', 'reduct', 'openwho', 'launch', 'multi-ti', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'response.', 'start', 'learn', 'today', '&amp;', 'ready4response:', '👉', 'https://t.co/hbffof0xkl', 'https://t.co/fgzy22rwu']


#### Inverted Index Construction

So the first step in building a text search engine is assembling an inverted index. An inverted index is a data structure that maps tokens to the documents they appear in. In this context, we can consider a token to simply be a word, so an inverted index, at its most basic, is just something that takes in a word, and returns to us a list of the documents it appears in.

In [65]:
def inverted_index(data, num_documents):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    data -- collection of tweets
    num_documents -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    index = defaultdict(list)
    tf = defaultdict(list)
    df = defaultdict(int)
    title_index = defaultdict(str)
    idf = defaultdict(float)
    
    for i in range(len(data)):
        tweet = data.get(f'{i}')
        page_id = i
        terms = preprocess(''.join(tweet.get('full_text'))) 
        title = tweet.get('full_text')
        title_index[page_id]=title

        ## As suggested in class, we are using a current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}
        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        current_page_index = {}
        for position, term in enumerate(terms):
            try:
                current_page_index[term][page_id].append(position)
            except:
                current_page_index[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        # normalized the term frequencies
        norm = 0
        for term, posting in current_page_index.items():
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)
        
        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            tf[term].append(np.round(len(posting[1])/norm,4))
            df[term] = df[term] + 1 
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        # Compute IDF
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)
            
    return index, tf, df, idf, title_index

In [66]:
num_documents = len(data)
index, tf, df, idf, title_index = inverted_index(data, num_documents)

In [69]:
#Eample
print("First 5 Index results for the term 'covid19': \n{}".format(index['covid19'][:5]))

First 5 Index results for the term 'covid19': 
[[1, array('I', [0])], [6, array('I', [8])], [8, array('I', [2])], [11, array('I', [7])], [13, array('I', [8])]]


# Querying the Index


#### Ranking

When searching in a search engine, we are interested in obtain the results sorted by relevance or by some other criteria.

In [70]:
def rank_documents(terms, docs, index, idf, tf, title_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
    
    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)
    query_terms_count = collections.Counter(terms)
    query_norm = la.norm(list(query_terms_count.values()))
    for termIndex, term in enumerate(terms):
        if term not in index: continue
        query_vector[termIndex]= query_terms_count[term]/query_norm * idf[term]
        for doc_index, (doc, postings) in enumerate(index[term]):
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    if len(result_docs) == 0: 
        print("No results found, try again")
        query = input()
        docs = search(query, index)
    return result_docs

#### Search

Enter queries to retrieved the top 10 relevant links based on the cosine similarity score (ltc.ltn schema)

In [71]:
def search(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = preprocess(query)
    docs = set()
    for term in query:
        try:                      
            term_docs=[posting[0] for posting in index[term]]
            docs |= set(term_docs)
        except:
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf, title_index)
    return ranked_docs

In [92]:
print("Insert your query (i.e.: covid19):\n")
query = input()
docs = search(query.lower(), index)
top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
  tweet = data.get(f'{d_id}')
  if tweet:
    user = tweet.get('user')
    entities = tweet.get('entities')
    hashtags = entities.get('hashtags')
    title = title_index[d_id]
    date = tweet.get('created_at')
    like = tweet.get('like')
    retweets = tweet.get('retweet_count')
    url = tweet.get('url')
    # if hashtags:
    #   title = hashtags[0]['text']
    print("Tweet_id = {} | Title = {} | Username: {} | Date: {} | Hashtags: {} | Likes: {} | Retweets: {} | Url: {}".format(d_id, title, user, date, hashtags, like, retweets, url))

Insert your query (i.e.: covid19):

computer science

Sample of 10 results out of 6 for the searched query:

Tweet_id = 999 | Title = @DrTedros "The WHO Hub for Pandemic &amp; Epidemic Intelligence is about: leveraging innovations in data science, harnessing the power of AI, quantum computing &amp; other cutting-edge technologies, &amp; fostering greater sharing of data &amp; information, between communities &amp; countries"-@DrTedros | Username: {'id': 14499829, 'id_str': '14499829', 'name': 'World Health Organization (WHO)', 'screen_name': 'WHO', 'location': 'Geneva, Switzerland', 'description': 'We are the #UnitedNations’ health agency - #HealthForAll.\n▶️ Always check our latest tweets on #COVID19 for updated advice/information.', 'url': 'https://t.co/wVulKuROWG', 'entities': {'url': {'urls': [{'url': 'https://t.co/wVulKuROWG', 'expanded_url': 'http://www.who.int', 'display_url': 'who.int', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 